# Extract tap data from building surface time history in VTK format
The procedure involves the following
1. Collect all the VTK files from their individual OpenFOAM time history folders into one folder with increasing index --- [this workbook]
2. Convert the pressure time history from .VTK to .CSV and save the time history in individual files and one points file --- [ParaView]
3. Map the data to the tap locations --- [this workbook]

In [68]:
import numpy as np
import os
import sys
import warnings
import pandas as pd

import wind
import foam
import windIO


Only edit the following box

In [69]:
collectVTK = False
mapToTap = False

# 'ttuE010_000.0', 'ttuE010_450.2', 'ttuE010_900.1', 'ttuE087_000.0', 'ttuE087_450.2', 'ttuE087_900.0'
caseName = 'ttuE087_900.0'

caseDir = r"D:/tempData_depot/simData_CandC/"+ caseName
vtkDir = r"I:/backup/graham/gra_20_12_13/fmGlbl/final/"+ caseName + r"/postProcessing/wallPressure"
vtkOutDir = caseDir + r"/postProcessing/wallPressure_VTK"
vtkName = 'p_building.vtk'
fldName = 'p'

if caseName in ['ttuE010_000.0', 'ttuE087_000.0']:
    tapCoordFile = r'D:/tempData_depot/simData_CandC/tapCoords3D_0deg.csv'
    print('AoA = 0 deg')
elif caseName in ['ttuE010_450.2', 'ttuE087_450.2']:
    tapCoordFile = r'D:/tempData_depot/simData_CandC/tapCoords3D_45deg.csv'
    print('AoA = 45 deg')
elif caseName in ['ttuE010_900.1', 'ttuE087_900.0']:
    tapCoordFile = r'D:/tempData_depot/simData_CandC/tapCoords3D_90deg.csv'
    print('AoA = 90 deg')


AoA = 90 deg


In [70]:
# timesStr = os.listdir(vtkDir)
# times = np.sort(np.asarray(timesStr,dtype=float))
# timesFile = caseDir + r"/postProcessing/wallPressure_times.npy"
# np.save(timesFile,times)

## Collect VTK files into one directory

In [71]:
if collectVTK:
    import shutil
    os.makedirs(vtkOutDir, exist_ok=True)

    vtkTimes = os.listdir(vtkDir)
    vtkTimesNum = np.asarray(vtkTimes,dtype=float)
    vtkTimesSorted = sorted(zip(vtkTimesNum,vtkTimes),key=lambda x: x[0])
    vtkTimesSorted = list(vtkTimesSorted)

    for v in vtkTimesSorted:
        srcFile = vtkDir+'/'+v[1]+'/'+vtkName
        newName = fldName+'_'+str(int(round(v[0]*10000,4))).zfill(7)+'.vtk'
        destFile = vtkOutDir+'/'+newName
        if os.path.isfile(destFile):
            continue
        try:
            shutil.copy(srcFile,destFile)
            print(srcFile+'-->'+newName)
        except:
            print('ERROR! Skipping '+srcFile)

## Map $p$ to tap locations

In [72]:
if mapToTap:
    tapCoords = pd.read_csv(tapCoordFile)
    tapCoords = tapCoords[['x','y','z']].to_numpy()

    wallPressure_CSV_Dir = caseDir + r"/postProcessing/wallPressure_CSV"

    p,newPts,distance = windIO.extractTapDataFromCSV(wallPressure_CSV_Dir,tapCoords,writeToFile=False,showLog=True)
    print(f"Point distance\t\t Avg.:{np.mean(distance)}\tMax:{np.max(distance)}")
    print(f"Shape of p: {p.shape}")

    outFile = caseDir + r"/postProcessing/wallPressure_pOfT_TNGtaps.npy"
    np.save(outFile, p)


## Merge separately extracted .csv files from paraview

In [73]:
# start = 5086
# dirr = r'D:/tempData_depot/simData_CandC/ttuE087_000.0/postProcessing/wallPressure_CSV/temp_CSV/'
# files = [filename for filename in os.listdir(dirr) if filename.startswith("p_")]
# numOrig = [int(filename.split('_')[1].split('.')[0])+start for filename in os.listdir(dirr) if filename.startswith("p_")]
# for file,id in zip(files,numOrig):
#     os.rename(dirr+file, dirr+'p_'+str(id)+'.csv')
# print(files)